In [3]:
import pymysql
import pandas as pd
import datetime
import numpy as np
import FinanceDataReader as fdr

In [66]:
def set_data(df,com_num = 0):  # panda, numpy, datetime, FinanceDataReader, konlpy, Counter
    # 일, 시간 나누기
    positive = ['수상','인기']
    negative = ['갑질','하락']
    df['date'] = df['date'].astype(str)
    df['date_d'] = df['date'].str[:-2]
    df['date_h'] = df['date'].str[-2:]
    # 타입을 데이트 타입으로 만듬
    df['date_d'] = pd.to_datetime(df['date_d'])
    
    df = df.sort_values(by='date_d') # 일기준으로 오름차순 정렬
    
    if com_num != 0:
    #     통합 데이터 활용시 업종코드 지정
        df = df[df['st_cd'] == com_num] # 해당 회사만 추출
        num = str(com_num).zfill(6) # 종목코드를 6자리로 만들어줌
        
    else:
        # 특정 업종만 할 때
        num = str(df['st_cd'].iloc[0]).zfill(6)
    
    # 널값 제거
    df = df.dropna()
    
    # ============== 날짜 조정
    ## 전일 15시 ~ 금일 15시로 날짜 조정
    after_market = ['15', '16', '17', '18', '19', '20', '21', '22', '23']
    
    for i in range(len(df)):
        if df['date_h'].iloc[i] in after_market:
            df['date_d'].iloc[i] += datetime.timedelta(1)
    
    # 주말 및 공휴일 데이터
    ### Holidays
    db = pymysql.connect(user='root',
                         passwd='1234',
                         host='3.35.70.166',
                         db='proj',
                         charset='utf8')

    cursor = db.cursor(pymysql.cursors.DictCursor)

    sql = "select * from holidays"
    cursor.execute(sql)
    result = cursor.fetchall()
    
    # DataFrame으로 변경
    holi = pd.DataFrame(result)
    
    # date 컬럼을 날짜 형식으로 변경
    holi['date'] = pd.to_datetime(holi['date'])
    
    
    ### ===================주말 및 공휴일 제외
    
    ## 주말 및 공휴일만 추출
    market_closed = holi[holi['holiday']=="O"].reset_index(drop=True)
    
    ## 휴장일 List 생성
    market_closed_list = list(market_closed['date'])
    
    # 주말 및 공휴일 제외
    while len(df[df['date_d'].isin(market_closed_list)]['date_d']) !=0:
        for i in df[df['date_d'].isin(market_closed_list)]['date_d'].index:
            df['date_d'][i] += datetime.timedelta(1)
     
    
    
    # 주식 데이터 가져오기
    start_date = str(df['date_d'].iloc[0])[:10].replace('-', '')
    end_date = str(df['date_d'].iloc[-1])[:10].replace('-', '')  # 뉴스 데이터 기준 최근 값까지의 주식 데이터
    
    stock = fdr.DataReader(num, start = start_date, end = end_date)
    
    # change 컬럼 판단 기준 close에서 수수료 보다 많은 비율이 올랐으면 1, 떨어졌으면 -1, 그 사이 0
    point = stock['Close'] * stock['Change']
    spli = stock['Close'] * 0.001  # 수수료가 0.1 % 라고 할 때
    
    stock['ud'] = np.where( point > spli, 1,
                          np.where(point < spli, -1, 0))
    
    stock = stock.reset_index() # 날짜 컬럼 가져오기
    
    s_1 = stock[['Date','ud']] # 필요한 컬럼만 가져오기
    
    s_1.columns = ['date_d','ud'] # 컬럼 명 초기화
    
    # 뉴스데이터와 주식데이터 merge
    df_1 = pd.merge(df,s_1,on='date_d')  # 영향받는 컬럼 기준으로 inner 조인 
    
    return df_1

In [68]:
# 연습
import pandas as pd
df = pd.read_csv('./df_매일경제_LG화학.csv')

In [69]:
set_data(df)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,st_n,st_cd,news,date,title,url,text,date_d,date_h,ud
0,LG화학,51910,매일경제,2018010118,中정부 전기차 면세 연장에 코스모화학 등 수혜株 주목,http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 중국발 수요 증가에 힘입어 새해 '전기차 랠리'에 대한 기대감이 확산되...,2018-01-02,18,1
1,LG화학,51910,매일경제,2018010210,"[신년사] 박진수 LG화학 부회장 ""사업구조·사업방식·조직문화 바꿔야""",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n박진수 LG화학 부회장. [사진 제공 = LG화학]\n 박진수 LG화학 ...,2018-01-02,10,1
2,LG화학,51910,매일경제,2018010204,"LG ,마곡사이언스파크 완공 R&D강화",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n지난해 미국 라스베이거스에서 열린 `CES 2017`에서 LG전자가 올레...,2018-01-02,04,1
3,LG화학,51910,매일경제,2018010204,"[다시 뛰는 재계] 차세대 먹을거리 차곡차곡 ""4차 산업혁명 우리가 주도""",http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n인천국제공항 제2터미널에서 항공기가 떠오르고 있다. 무술년(戊戌年)을 맞...,2018-01-02,04,1
4,LG화학,51910,매일경제,2018010216,"2년 연속 최대 이익 예상되는 화학업계, 올해도 상승세 이어질까?",http://news.mk.co.kr/newsRead.php?year=2018&no...,화학업계가 지난해까지 2년 연속 사상 최대 영업이익을 갈아 치울 것으로 예상되는 가...,2018-01-03,16,-1
...,...,...,...,...,...,...,...,...,...,...
3778,LG화학,51910,매일경제,2021091004,"삼성증권, 韓증시 선도 BBIG투자…업계 첫 2배 레버리지",http://news.mk.co.kr/newsRead.php?no=873455&ye...,◆ 증권사 추천상품 ◆\r\n\n\n 삼성증권이 'KRX 2차전지 K-뉴딜 지수'와...,2021-09-10,04,-1
3779,LG화학,51910,매일경제,2021091004,"KB증권 '레버리지 2차전지 K-뉴딜 ETN', 한국대표 2차전지株에 레버리지 투자",http://news.mk.co.kr/newsRead.php?no=873450&ye...,◆ 배당주 투자 전략 / 증권사 추천상품 ◆\r\n\n\n 기후변화가 전 세계인의...,2021-09-10,04,-1
3780,LG화학,51910,매일경제,2021091017,"""애플 주문 쏟아질라, 얼른 사자""…LG전자 이노텍 주가 들썩들썩",http://news.mk.co.kr/newsRead.php?no=876664&ye...,\n\n\n 애플의 자율주행 전기차인 애플카가 10일 한국 증시를 또다시 흔들었다....,2021-09-13,17,1
3781,LG화학,51910,매일경제,2021091214,"""10만원에 페이스북 아마존 애플 넷플릭스 구글 다 살 수 있다?""",http://news.mk.co.kr/newsRead.php?no=878988&ye...,"◆ 고가주식 쪼개서 매매 ◆ \r아마존, 알파벳, LG생활건강 등 주당 100만원이...",2021-09-13,14,1


In [1]:
def giro_dic(df,num = 0):
    
    set_data(df, num)
    # =================== nlp
    # 1. 불용어 처리
    # 2. 제목, 텍스트 --> 명사, 형용사 추출
    
    # 태그 삭제        
    # \n \t 삭제
    df['text'] = df['text'].str.replace('[\n|\t|\r]','')
    
    
    # positive --> change == 1

    df_pos = df_1[df_1['ud'] == 1]['title']
    
    okt = Okt()
    
    for i in df_pos:
        ppp = okt.nouns(i)
        positive = positive + ppp
    
    # 본문 토크나이징으로 감성어휘 추가
    
    df_pos_t = df_1[df_1['change'] == 1]['text']
    for i in df_pos_t:
        ppp = okt.nouns(i)
        positive = positive + ppp
        
    # 제목 토크나이징을 통해서 감성어휘 추가

    # negative --> change == -1
    
    df_na = df_1[df_1['change'] == -1]['title']

    for i in df_na:
        ppp = okt.nouns(i)
        negative = negative + ppp
    
    df_na_t = df_1[df_1['change'] == -1]['text']
    
    # 본문
    
    for i in df_na_t:
        ppp = okt.nouns(i)
        negative = negative + ppp
    
    
    pos_cnt = Counter(negative) # 갯수 확인
    nega_cnt = Counter(negative)
     
    positive_r = list(set(positive)) # 중복 제거
    negative_r = list(set(negative))

    # 자체 제작 기반 감성점수도출
    
    df_1['score'] = 0
    df_1['score_b'] = 0.1 # 비율이라 타입을 float로 맞춰줌
    
    for i in range(len(df_1)):
        score = 0
        num = okt.nouns(df_1['text'].iloc[i])
        if len(num) == 0:
            continue
            
        for j in num:
            if j in positive_r:
                score+=1
            if j in negative_r:
                score-=1
            else:
                score+=0
        df_1['score'].iloc[i] = score
        df_1['score_b'].iloc[i] = score/len(num)
    
    
    
    return df_1, pos_cnt, nega_cnt, positive_r, negative_r

## ---------------- db----------------

In [2]:
import pymysql
import pandas

db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='3.35.70.166', 
    db='proj', 
    charset = 'utf8'
)

cursor = db.cursor(pymysql.cursors.DictCursor)

sql = "select * from news_craw_005930 where length(date) = 10"  # date 널값 제외
cursor.execute(sql)

result = cursor.fetchall()

# 데이터 프레임으로 변경
df = pandas.DataFrame(result)

## --------------------실습 ----------------------

## 함수 실습

In [3]:
# panda, numpy, datetime, FinanceDataReader, konlpy, Counter
import pandas as pd
import numpy as np
import datetime
import FinanceDataReader as fdr
from konlpy.tag import Okt
from collections import Counter

In [4]:
df_a, pos_cnt, nega_cnt, positive_r, negative_r =  giro_dic(df, 5930)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
df_a

,st_n,st_cd,news,date,title,url,text,date_d,date_h,date_i,change,score,score_b
0,삼성전자,005930,매일경제,2020010113,삼성 반도체 공장 1분 정전…수십억 피해 예상,http://news.mk.co.kr/newsRead.php?no=1618&year...,[사진 제공: 연합뉴스] 삼성전자 경기 화성 사업장에 지난 31일 정전이 발생해 일...,2020-01-01,13,2020-01-02,-1,-2,-0.011299
1,삼성전자,005930,매일경제,2020010113,삼성 반도체 공장 1분 정전…수십억 피해 예상,http://news.mk.co.kr/newsRead.php?no=1618&year...,[사진 제공: 연합뉴스] 삼성전자 경기 화성 사업장에 지난 31일 정전이 발생해 일...,2020-01-01,13,2020-01-02,-1,-2,-0.011299
2,삼성전자,005930,매일경제,2020010104,무르익는 5G·폴더블폰…4년만에 반등 예감,http://news.mk.co.kr/newsRead.php?no=327&year=...,◆ 2020 경제기상도 / 업종별 전망 (스마트폰) ◆\r 올해는 5세대(5G) 이...,2020-01-01,04,2020-01-02,-1,-2,-0.009302
3,삼성전자,005930,매일경제,2020010104,의류건조기·식기세척기…新가전 열풍,http://news.mk.co.kr/newsRead.php?no=330&year=...,◆ 2020 경제기상도 / 업종별 전망 (가전) ◆\rLG전자가 올해 9월 독일 베...,2020-01-01,04,2020-01-02,-1,-4,-0.014815
4,삼성전자,005930,매일경제,2020010118,삼성전자 `QLED 8K TV` 美소비자기술협회 인증 획득,http://news.mk.co.kr/newsRead.php?no=2206&year...,삼성전자가 올해 출시하는 8K(4K UHD의 4배 화질) TV가 미국 소비자기술협회...,2020-01-01,18,2020-01-02,-1,-3,-0.013953
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4847,삼성전자,005930,매일경제,2021090117,"갤폴드3 플립3 무섭게 팔린다…삼성보다 더 신났다, 누구?",http://news.mk.co.kr/newsRead.php?no=845977&ye...,삼성전자의 폰더블폰 신모델(Z폴더3·플립3)이 초기 수요 흥행에 성공하면서 관련 부...,2021-09-01,17,2021-09-02,-1,0,0.000000
4848,삼성전자,005930,매일경제,2021090117,"제일기획, 하이브랩 투자…디지털 경쟁력 키운다",http://news.mk.co.kr/newsRead.php?no=846009&ye...,제일기획(대표 유정근)이 '하이브랩'과 손잡고 디지털 경쟁력 강화에 나선다. 제일기...,2021-09-01,17,2021-09-02,-1,-2,-0.017857
4849,삼성전자,005930,매일경제,2021090108,"삼성전자, 고려대와 손잡고 친환경 생활가전 솔루션 개발한다",http://news.mk.co.kr/newsRead.php?no=842660&ye...,31일 서울 성북구 소재 고려대 공학관에서 열린 삼성전자-고려대 산학협력 협약식에서...,2021-09-01,08,2021-09-02,-1,-13,-0.057778
4850,삼성전자,005930,매일경제,2021090117,"""저가매수도 시기상조""…악소리 나는 배터리 게임 대장주를 어찌할꼬",http://news.mk.co.kr/newsRead.php?no=846038&ye...,최근 국내 증시에서 개별 종목 장세가 연출되는 가운데 악재가 발생할 때 해당 종목...,2021-09-01,17,2021-09-02,-1,-13,-0.034483


## 실습 1

In [74]:
df.head(1)

,st_n,st_cd,news,date,title,url,text,date_d,date_h
0,삼성전자,005930,매일경제,2021090117,"""꿈의 격차 줄이자"" 삼성, 교육 사다리 놓는다",http://news.mk.co.kr/newsRead.php?no=845994&ye...,\n\n\n2015년 8월 광주 전남대에서 열린 드림클래스 여름 캠프를 방문한 이재...,2021-09-01,17


In [70]:
df['text'].str.replace('[\n|\t]','')

4701    [사진 제공: 연합뉴스] 삼성전자 경기 화성 사업장에 지난 31일 정전이 발생해 일...
2443    [사진 제공: 연합뉴스] 삼성전자 경기 화성 사업장에 지난 31일 정전이 발생해 일...
6180    ◆ 2020 경제기상도 / 업종별 전망 (스마트폰) ◆\r 올해는 5세대(5G) 이...
5617    ◆ 2020 경제기상도 / 업종별 전망 (가전) ◆\rLG전자가 올해 9월 독일 베...
4566    삼성전자가 올해 출시하는 8K(4K UHD의 4배 화질) TV가 미국 소비자기술협회...
                              ...                        
4       삼성전자의 폰더블폰 신모델(Z폴더3·플립3)이 초기 수요 흥행에 성공하면서 관련 부...
270     제일기획(대표 유정근)이 '하이브랩'과 손잡고 디지털 경쟁력 강화에 나선다. 제일기...
1       31일 서울 성북구 소재 고려대 공학관에서 열린 삼성전자-고려대 산학협력 협약식에서...
166      최근 국내 증시에서 개별 종목 장세가 연출되는 가운데 악재가 발생할 때 해당 종목...
0       2015년 8월 광주 전남대에서 열린 드림클래스 여름 캠프를 방문한 이재용 삼성전자...
Name: text, Length: 6182, dtype: object

In [5]:
# pip install xlrd
# !pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 243 kB 1.7 MB/s eta 0:00:01


In [19]:
df_hundai = pd.read_excel('./news_craw_hyundai.xlsx', engine = 'openpyxl')

In [20]:
df_hundai = df_hundai.dropna()

In [22]:
df_hundai['date'] = df_hundai['date'].astype(int)

In [23]:
df_hundai.head()

,Unnamed: 0,st_n,st_cd,news,date,title,url,text
0,0,현대차,5380,매일경제,2021090109,"""어서와 이런 깜찍한 SUV는 처음이지?""…현대차, 캐스퍼 베일 벗었다",http://news.mk.co.kr/newsRead.php?no=842734&ye...,"\n\n\n현대차, 신규 엔트리 SUV `캐스퍼` 외장 이미지 최초 공개 [사진제공..."
1,1,현대차,5380,매일경제,2021090111,“메타버스로 경험하는 현대차”…현대 모빌리티 어드벤처 최초 공개,http://news.mk.co.kr/newsRead.php?no=843907&ye...,\n\n\n“메타버스로 경험하는 현대차”…현대 모빌리티 어드벤처 최초 공개 [사진제...
2,2,현대차,5380,매일경제,2021090117,"""이런 앙증맞은 SUV 딱 내 취향""…현대차 캐스퍼 나온다",http://news.mk.co.kr/newsRead.php?no=846010&ye...,\n\n\n현대차가 연내 출시할 경형 SUV `캐스퍼`. [사진 제공 = 현대차]...
3,3,현대차,5380,매일경제,2021090116,"현대차, 8월 국내외서 총 29만4591대 판매…전년比 7.6%↓",http://news.mk.co.kr/newsRead.php?no=845733&ye...,현대자동차가 지난 8월 국내외 시장에서 나란히 전년 동월 대비 판매 감소를 기록했다...
4,4,현대차,5380,매일경제,2021083118,"""꿈이 현실로""…'무인 자율주행' 현대차 아이오닉5 내후년 북미 대륙 달린다",http://news.mk.co.kr/newsRead.php?no=841961&ye...,\n\n\n\n▶ 여기를 누르시면 크게 보실 수 있습니다\n 현대자동차그룹의 첫 전...


In [24]:
df_hundai.drop('Unnamed: 0',1, inplace= True)

In [25]:
df_hundai.columns = ['st_n', 'st_cd', 'news', 'n_date', 'title', 'url', 'text']

In [26]:
df_hundai.head(1)

,st_n,st_cd,news,n_date,title,url,text
0,현대차,5380,매일경제,2021090109,"""어서와 이런 깜찍한 SUV는 처음이지?""…현대차, 캐스퍼 베일 벗었다",http://news.mk.co.kr/newsRead.php?no=842734&ye...,"\n\n\n현대차, 신규 엔트리 SUV `캐스퍼` 외장 이미지 최초 공개 [사진제공..."


AttributeError: Can only use .str accessor with string values!

In [35]:
df_a.head()

,st_n,st_cd,news,n_date,title,url,text,date_d,date_h,date_i,change,score,score_b
0,삼성전자,5930,매일경제,2021080121,"""외국인 기관 9조 매물 받아냈다""…'사긴 사는데 꺼림칙' 개미 돌파구는…",http://news.mk.co.kr/newsRead.php?no=743682&ye...,"29일 서울 중구 을지로 하나은행 본점 딜링룸 현황판에 코스피 3,242.65포인트...",2021-08-01,21,2021-08-02,1,24,0.048290
1,삼성전자,5930,매일경제,2021080106,"""재테크하려면 지금 당장 가입하라""…증권사 전화 불나게 만든 상품은?",http://news.mk.co.kr/newsRead.php?no=741674&ye...,[사진 = 이승환 기자] 다양한 금융상품에 투자할 수 있고 세제 혜택도 있어 '만능...,2021-08-01,06,2021-08-02,1,44,0.061798
2,삼성전자,5930,매일경제,2021080117,지난달 개미 9조 샀지만…수익은 지지부진,http://news.mk.co.kr/newsRead.php?no=742856&ye...,개인투자자가 지난달 반도체와 전기차 관련주에 집중 투자했지만 손실을 봤던 것으로 ...,2021-08-01,17,2021-08-02,1,15,0.043353
3,삼성전자,5930,매일경제,2021080111,"""그때 내가 왜 샀지"" 동학개미…지난달 9조 순매수, 줄줄이 마이너스",http://news.mk.co.kr/newsRead.php?no=742016&ye...,지난달 30일 오후 서울 중구 하나은행 본점 딜링룸에 이날 마감된 코스피가 표시돼 ...,2021-08-01,11,2021-08-02,1,14,0.039660
4,삼성전자,5930,매일경제,2021080117,[포토] 판매량 증가하는 위생가전,http://news.mk.co.kr/newsRead.php?no=742870&ye...,세컨드 가전의 인기는 폭발적이다.자랜드에 따르면 작년 1월부터 12월 20일까지 ...,2021-08-01,17,2021-08-02,1,5,0.084746


In [36]:
pos_cnt

(Counter({'갑질': 1,
          '하락': 164,
          '하루': 25,
          '평균': 33,
          '가입자': 13,
          '배': 56,
          '비': 48,
          '과세': 14,
          '뭉칫돈': 1,
          '국민': 51,
          '통장': 3,
          '대학생': 7,
          '가장': 86,
          '취업': 60,
          '회사': 54,
          '위': 259,
          '삼성': 755,
          '전자': 828,
          '장기': 36,
          '금리': 21,
          '주춤': 7,
          '장주': 7,
          '다시': 45,
          '힘': 25,
          '넷마블': 6,
          '선호': 6,
          '게임': 39,
          '기업': 286,
          '추세': 23,
          '반등': 18,
          '달': 134,
          '회복': 23,
          '인재': 20,
          '입': 6,
          '조직': 16,
          '정비': 2,
          '진출': 14,
          '본격': 29,
          '채비': 1,
          '증권': 126,
          '어디': 3,
          '투자': 317,
          '갤럭시': 441,
          '노트': 22,
          '출시': 89,
          '팩': 55,
          '행사': 66,
          '때': 64,
          '제품': 94,
          '하이닉스': 163,
   

In [37]:
nega_cnt

Counter({'갑질': 1,
         '하락': 164,
         '하루': 25,
         '평균': 33,
         '가입자': 13,
         '배': 56,
         '비': 48,
         '과세': 14,
         '뭉칫돈': 1,
         '국민': 51,
         '통장': 3,
         '대학생': 7,
         '가장': 86,
         '취업': 60,
         '회사': 54,
         '위': 259,
         '삼성': 755,
         '전자': 828,
         '장기': 36,
         '금리': 21,
         '주춤': 7,
         '장주': 7,
         '다시': 45,
         '힘': 25,
         '넷마블': 6,
         '선호': 6,
         '게임': 39,
         '기업': 286,
         '추세': 23,
         '반등': 18,
         '달': 134,
         '회복': 23,
         '인재': 20,
         '입': 6,
         '조직': 16,
         '정비': 2,
         '진출': 14,
         '본격': 29,
         '채비': 1,
         '증권': 126,
         '어디': 3,
         '투자': 317,
         '갤럭시': 441,
         '노트': 22,
         '출시': 89,
         '팩': 55,
         '행사': 66,
         '때': 64,
         '제품': 94,
         '하이닉스': 163,
         '집중': 42,
         '매수': 64,
         '외국인':

In [38]:
positive_r

['중이',
 '이동식',
 '직계',
 '앤디',
 '전공',
 '무단',
 '가파른',
 '단절',
 '트렌드',
 '샤넬',
 '지표',
 '일도',
 '명',
 '휴렛팩커드',
 '해지',
 '준공',
 '대악',
 '강도',
 '고음',
 '채',
 '진행형',
 '일로',
 '이재철',
 '비',
 '아치',
 '지난해',
 '견조할',
 '급등',
 '강조',
 '플래트너',
 '중요성',
 '이상인',
 '세션',
 '보완',
 '첨단',
 '지출',
 '벌이',
 '마지막',
 '감행',
 '팬',
 '금성사',
 '로서',
 '전성기',
 '리즈',
 '간편',
 '탈환',
 '협성회',
 '의견',
 '이사',
 '맨',
 '사택',
 '매트',
 '이식',
 '금전',
 '황',
 '즉각',
 '유럽의회',
 '법인',
 '보급',
 '근로',
 '어플',
 '세금',
 '물',
 '리가',
 '조언',
 '휴업',
 '박석현',
 '사용',
 '찬반',
 '호응',
 '종조부',
 '리드',
 '도약',
 '에릭슨',
 '보안',
 '편성',
 '질문',
 '게티',
 '텐센트',
 '보름',
 '사이니',
 '눈앞',
 '금도',
 '이병철',
 '재산',
 '폭',
 '계기',
 '유동성',
 '연중',
 '이공',
 '장학',
 '상무',
 '장마',
 '법적',
 '신용',
 '내용',
 '프라다폰',
 '우리나라',
 '후기',
 '무슨',
 '블랙',
 '여름',
 '사업',
 '다시',
 '어닝',
 '지대',
 '가수',
 '투자자',
 '바',
 '적정',
 '가량',
 '무선통신',
 '치',
 '조건',
 '만',
 '영상',
 '그동안',
 '정도',
 '멘탈',
 '구도',
 '장점',
 '선수',
 '진두',
 '몸',
 '통계',
 '항셍지수',
 '특화된',
 '배치',
 '식품',
 '부쩍',
 '정말',
 '부사',
 '연초',
 '썸머',
 '스캔',
 '대중화',
 '저축',
 '저택',
 '문구',


In [39]:
negative_r

['나은',
 '가파른',
 '만족도',
 '지표',
 '젤로',
 '명',
 '보일러',
 '수렴',
 '강도',
 '정거장',
 '견조할',
 '재무부',
 '이상인',
 '박람회',
 '고대',
 '로서',
 '간편',
 '점일',
 '웅장',
 '인근',
 '좌측',
 '보급',
 '본딩',
 '황',
 '캐슬',
 '광택',
 '상담사',
 '호응',
 '규칙',
 '도약',
 '점진',
 '편성',
 '김현정',
 '금도',
 '이진우',
 '오찬',
 '분양',
 '스타일',
 '투자자',
 '뉴스',
 '조건',
 '영상',
 '삭제',
 '감정',
 '부사',
 '과징금',
 '이충',
 '고향',
 '대식',
 '선도',
 '제자리',
 '작용',
 '사용자',
 '강의',
 '이쯤',
 '한마디',
 '유려',
 '산화물',
 '골드',
 '환경오염',
 '연구원',
 '구축',
 '괄목상대',
 '전보',
 '위트',
 '호도',
 '눈길',
 '수조',
 '상가',
 '부인',
 '지방',
 '차별',
 '건물',
 '사고',
 '창업',
 '대체',
 '창달',
 '모리',
 '봉착',
 '팝업',
 '구속영장',
 '변동성',
 '레츠고',
 '일류',
 '흥행',
 '활동',
 '도모',
 '승계',
 '해소',
 '전일',
 '지적재산',
 '김치냉장고',
 '바이오',
 '찌',
 '서비스',
 '경제부총리',
 '채비',
 '제품',
 '성장동력',
 '구가',
 '엔젤',
 '허가',
 '이명희',
 '웁니',
 '정은경',
 '마이크로미터',
 '백화점',
 '조세',
 '제조업체',
 '전기레인지',
 '더니',
 '기프티콘',
 '캠페인',
 '확답',
 '탓',
 '드라이브인',
 '발판',
 '승인',
 '맥쿼리인프라',
 '풀무원',
 '접',
 '발간',
 '잠재력',
 '오피스',
 '김순',
 '자사',
 '남부',
 '회안',
 '서명',
 '마벨',
 '가로',
 '다수',
 '부작용',
 '웨딩',
 '